In [46]:
# Standard imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import os
from IPython.display import display
from ipywidgets import interact, widgets
from PIL import Image
import numpy as np
import sys
import time
import shutil
import os
import matplotlib.pyplot as plt
import pandas
import networkx as nx

In [2]:
# Local clone
! git clone https://github.com/nanopiero/annotation.git
import sys
sys.path.append('annotation')


Cloning into 'annotation'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 54 (delta 29), reused 28 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 253.53 KiB | 6.50 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [10]:
# Import the functions we need to sort a sequence :
import importlib
import src.utils as utils
import src.poset_mergesort_functions_per_level  as pmf

In [ ]:
# Prepare a sequence to sort :

In [11]:
! pip install huggingface_hub
from huggingface_hub import hf_hub_download

In [ ]:
import os
import zipfile
from huggingface_hub import hf_hub_download

# Step 1: Download the zip file using hf_hub_download
local_zip = hf_hub_download(repo_id="nanopiero/weow", filename="webcam_images.zip")

# Step 2: Unzip the contents
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall('webcam_images')

In [ ]:
# List all .jpg images in the directory
image_dir = 'webcam_images'
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')], key=lambda x: int(x.split('.')[0]))

def show_image(index):
    img_path = os.path.join('webcam_images', image_files[index])
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()

# Use a slider to scroll through the images
interact(show_image, index=widgets.IntSlider(min=0, max=len(image_files)-1, step=1, description="Image Index"));


In [59]:
# Prepare the folder for the sequence "0000":
! mkdir -p dataset/0000/labels_ord dataset/0000/images
! cp webcam_images/* dataset/0000/images/
# ! ls dataset/0000/images

In [60]:
# Get the dictionnary of snow_cover levels
# resulting from the image wise annotation step:
# (here, the dict is arbitrarily set to the level '0' for all images)
lbls = {n: {'sequence':'0000', 'levelsc':'0'} for n in ls('dataset/0000/images')}

In [61]:
# Config:
critere = 'sc'
param = 'sc'
mode = 'surface'
modes = [mode]
subgroup = ''
times = ['']

grounds = [r'snow_ground', r'snow_road', r'white_road', r'snow_ground_dry_road']

# path to the folder that contains sequences
dir_splitted = 'dataset'

# path to a tmp dir where sequences are sorted:
! mkdir -p tmp/images

In [62]:
# Define the list of sequences to label and the dict seq2images
sequences = sorted({lbls[name]['sequence'] for name in lbls})

# Create a dictionary mapping sequences to image sets
seq2images = {
    sequence: {name for name in lbls if lbls[name]['sequence'] == sequence}
    for sequence in sequences
}

# List sequences that have more than one associated image
sequences_to_sort = [seq for seq, images in seq2images.items() if len(images) > 1]

print(sequences_to_sort, len(sequences_to_sort), len(sequences))


['0000'] 1 1


In [63]:
# generate new graphs dg, ug, eg:
graphs = utils.get_new_graphs(lbls)
tdg, tug, teg = utils.get_sdg_from_levels(graphs, lbls, param) # no supplementary edges

0000
set()


In [64]:
# Add previous comparisons (stored in labels_ord) & Cleaning : one must not have cycles
graphs= (tdg, tug, teg)
tdg, tug, teg = utils.fill_graph_from_splitted(graphs, dir_splitted, ext='sc')
print(len(tdg.edges), len(tug.edges), len(teg.edges))


#washing
two_cycles = set(tdg.edges).intersection(utils.invert_edges(set(tdg.edges)))
two_cycles = utils.edges_and_inverted_edges(two_cycles)
print(two_cycles)
#%%
print(str(len(two_cycles)) + ' 2-cycles')
tdg.remove_edges_from(two_cycles)
tdg = nx.transitive_closure(tdg)
two_cycles = set(tdg.edges).intersection(utils.invert_edges(set(tdg.edges)))
two_cycles = utils.edges_and_inverted_edges(two_cycles)
print(str(len(two_cycles)) + ' 2-cycles after washing')


vs03 = utils.edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3')
vs03eg = {edge for edge in vs03 if edge in teg.edges}
tdg.remove_edges_from(vs03)
tug.remove_edges_from(vs03)
teg.remove_edges_from(vs03eg)

tdg = nx.transitive_closure(tdg)

vs03 = utils.edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3 after washing')
print(len(tdg.edges), len(tug.edges), len(teg.edges))

nothing in sequence: 0000
nothing in sequence: 0
0 0 0
set()
0 2-cycles
0 2-cycles after washing
0 0vs3
0 0vs3 after washing
0 0 0


In [80]:
! cd annotation ; git pull ; cd ..
# Reload each module if needed
importlib.reload(utils)
importlib.reload(pmf)

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1012 bytes | 337.00 KiB/s, done.
From https://github.com/nanopiero/annotation
   b489266..57aef27  main       -> origin/main
Updating b489266..57aef27
Fast-forward
 src/poset_mergesort_functions_per_level.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


<module 'src.poset_mergesort_functions_per_level' from '/content/annotation/src/poset_mergesort_functions_per_level.py'>

In [ ]:
for i, sequence in enumerate(sequences_to_sort):
    print('sequence: ' + str(sequence), i)
    root_dataset = dir_splitted
    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    root_cs = join(dataset, r'labels_ord')

    if not isdir(root_cs):
        os.mkdir(root_cs)

    # init frame_lbls and dic of subgroup
    frame_lbls = {}
    for name in os.listdir(images_dataset):
            frame_lbls[name] = lbls[name]

    dic_of_subgroups = {}
    dic_of_subgroups[''] = [name for name in frame_lbls]

    labeling_root= "tmp"
    temp_images_dir= "tmp/images"
    name_poset_sc = r'poset_sc.pickle'

    #%def graph paths
    graphs_names = [os.path.join(graph + '_sc' +r'.gpickle') for graph in ["dg","ug","eg"]]
    graphs_paths = [os.path.join(root_cs, graph_name) for graph_name in graphs_names]

    #def nodes
    nodes = sorted(dic_of_subgroups[subgroup])
    print('labelling of ' + str(len(nodes)) + ' nodes')

    #clean images_dir
    for name in os.listdir(temp_images_dir):
        os.remove(os.path.join(temp_images_dir,name))

    #copy in the temp image dir
    for node in nodes:
        shutil.copy(os.path.join(dataset, 'images', node), os.path.join(temp_images_dir))

    # if graphs_names[0] not in os.listdir(root_cs):
    print("need to init graphs")
    # restrict tdg, tug, teg:
    dg = tdg.subgraph(nodes).copy()
    ug = tug.subgraph(nodes).copy()
    eg = teg.subgraph(nodes).copy()

    #save the graphs
    utils.write_gpickle(dg, graphs_paths[0])
    utils.write_gpickle(ug, graphs_paths[1])
    utils.write_gpickle(eg, graphs_paths[2])

    # else:
    #     pass


    kwargs = {'root_cs': root_cs,
              'critere':  'sc',
              'subgroup': subgroup,
              'mode' : '',
              'graphs_paths': graphs_paths,
              'images_dir' : temp_images_dir
              }

    decomposition = pmf.labelling_mode(lbls, **kwargs)

In [86]:
name1 = 'dataset/0000/images/0.jpg'
name2 = 'dataset/0000/images/1.jpg'


In [82]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from ipywidgets import interact

def compare(lbls, name1, name2, mode, images_dir, critere, **kwargs):
    image_path1 = os.path.join(images_dir, name1)
    image_path2 = os.path.join(images_dir, name2)

    im1 = Image.open(image_path1).resize((1000, 1000))
    im2 = Image.open(image_path2).resize((1000, 1000))

    def show_image(image_num):
        plt.figure(figsize=(10, 10))
        if image_num == 0:
            plt.title(f'Image 0: {name1[-10:-4]} {lbls[name1]["level" + critere]}')
            plt.imshow(im1)
        else:
            plt.title(f'Image 1: {name2[-10:-4]} {lbls[name2]["level" + critere]}')
            plt.imshow(im2)
        plt.axis('off')
        plt.show()

    interact(show_image, image_num=(0, 1))

In [ ]:
compare(lbls, '0.jpg', '1.jpg', '', 'dataset/0000/images', 'sc')

In [ ]:
#clean:
#00010268_0_20131114_081603.jpg
ntr = ['00010657_7002_20130721_180838.jpg']

clean_nodes(ntr, **kwargs)

#%%
path2 = os.path.join(root_cs, 'decomposition_vvday.pickle')
os.remove(path2)

In [ ]:
#for name in os.listdir(splitted_dir):
#    path = os.path.join(splitted_dir,name,'labels_ord')
#    if 'decomposition_snowday_surface.pickle' in os.listdir(path):
#        path2 = os.path.join(path, 'decomposition_snowday_surface.pickle')
#        os.remove(path2)

In [ ]:
# Autres éléments dans lesquels piquer évzntuellement :


def get_dg2vvday(root_cs):

    path_of_dg2 = os.path.join(root_cs, "poset_vvday" + r".gpickle" )
    dg2 = nx.read_gpickle(path_of_dg2)
    return dg2

def save_dg2vvday(dg2, root_cs):

    path_of_dg2 = os.path.join(root_cs, "poset_vvday" + r".gpickle" )
    nx.write_gpickle(dg2, path_of_dg2)
    print("graph dg2 saved")

def save_graphs(graphs, graphs_paths, **kwargs):
    dg, ug, eg = graphs
    nx.write_gpickle(dg, graphs_paths[0])
    nx.write_gpickle(ug, graphs_paths[1])
    nx.write_gpickle(eg, graphs_paths[2])
    print("graphs saved")

doublons  =[]
for i, sequence in enumerate(sequences_to_sort):  #[-42:-41]:

    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')


    for name in os.listdir(images_dataset):
        if 'Copie' in name:
            print('sequence: ' + str(sequence), i, name)

            correct_name = name.split(' - ')[0] + '.jpg'
            level = copy.deepcopy(corrected_lbls[name])


            print(correct_name, level, correct_name in os.listdir(images_dataset))

            if correct_name in os.listdir(images_dataset):
                doublons.append(name)

            else:
                corrected_lbls[correct_name] = level
#
            del corrected_lbls[name]

# Vérification:
for name in corrected_lbls:
    if 'Copie' in name:
        print(name)

#%% correction des graphs:
for i, sequence in enumerate(sequences_to_sort):
    modify_graph = False
    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')
    name_poset_vv = r'poset_vv.pickle'
    name_poset_snowsurface =r'poset_snow_surface.pickle'
    name_poset_snowheight = r'poset_snow_height.pickle'

    #%def graph paths
    suffixe = get_suffixe(critere,subgroup,mode)
    graphs_names = [os.path.join(graph + suffixe+r'.gpickle') for graph in ["dg","ug","eg"]]
    graphs_paths = [os.path.join(root_cs, graph_name) for graph_name in graphs_names]


    for name in os.listdir(images_dataset):
        if 'Copie' in name:
            modify_graph = True

    if modify_graph:

        dg, ug, eg = get_graphs(graphs_paths)
        dg2 = get_dg2vvday(root_cs)

        for name in os.listdir(images_dataset):
            if 'Copie' in name:

                if name in doublons:
                    print(name + ' was in doublons')
                    dg.remove_node(name)
                    ug.remove_node(name)
                    eg.remove_node(name)
                    dg2.remove_node(name)
                else:
                    correct_name = name.split(' - ')[0] + '.jpg'
                    mapping = {name:correct_name}
                    print(sequence, name, len(dg.nodes))
                    dg = nx.relabel_nodes(dg, mapping)
                    ug = nx.relabel_nodes(ug, mapping)
                    eg = nx.relabel_nodes(eg, mapping)
                    dg2 = nx.relabel_nodes(dg2, mapping)
                    print(name, len(dg.nodes))

        save_graphs((dg, ug, eg), graphs_paths)
        save_dg2vvday(dg2, root_cs)

#%% Vérification:

for i, sequence in enumerate(sequences_to_sort):
    modify_graph = False
    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')

    #%def graph paths
    suffixe = get_suffixe(critere, subgroup, mode)
    graphs_names = [os.path.join(graph + suffixe + r'.gpickle') \
                    for graph in ["dg","ug","eg"]]
    graphs_paths = [os.path.join(root_cs, graph_name) \
                    for graph_name in graphs_names]




    dg, ug, eg = get_graphs(graphs_paths)
    dg2 = get_dg2vvday(root_cs)

    for g in [dg, ug, eg, dg2]:
        for n in g.nodes:
            if "Copie" in n:
                print(n)

#%% renommer les images
for i, sequence in enumerate(sequences_to_sort):  #[-42:-41]:

    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')


    for name in os.listdir(images_dataset):
        if 'Copie' in name:
            print('sequence: ' + str(sequence), i, name)

            correct_name = name.split(' - ')[0] + '.jpg'


            if name in doublons:
                print("delete name")
                os.remove(join(images_dataset, name))
            else:
                src = join(images_dataset, name)
                dst = join(images_dataset, correct_name)
                print("move " + src + " to " + dst)
                shutil.move(src,dst)


#%% Vérif que correct_lbls contient toutes les images:
for i, sequence in enumerate(sequences_to_sort):  #[-42:-41]:

    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')

    seq_names = set([n for n in corrected_lbls if corrected_lbls[n]['sequence'] == sequence])

    if not seq_names == set(os.listdir(images_dataset)):
        print(seq)

#%% sauvegarde de corrected_labels:

label_path2 = os.path.join(root,
                    r'labels_imagewise_BAMOSvv_corr2.pickle')


with open(label_path2, 'wb') as f:
    pickle.dump(corrected_lbls, f)


#%% Fabrication des ug2:
def save_ug2vvday(ug2, root_cs):

    path_of_ug2 = os.path.join(root_cs, "posetbar_vvday" + r".gpickle" )
    nx.write_gpickle(ug2, path_of_ug2)
    print("graph ug2 saved")



counter = 0

for i, sequence in enumerate(sequences_to_sort):
    modify_graph = False
    root_dataset = dir_splitted
    #cam = r"nancy2"


    dataset = os.path.join(root_dataset, str(sequence))
    images_dataset = join(dataset, 'images')
    labels_dataset = join(dataset, 'labels')
    root_cs = join(dataset,r'labels_ord')
    name_poset_vv = r'poset_vv.pickle'
    name_poset_snowsurface =r'poset_snow_surface.pickle'
    name_poset_snowheight = r'poset_snow_height.pickle'

    #%def graph paths
    suffixe = get_suffixe(critere,subgroup,mode)
    graphs_names = [os.path.join(graph + suffixe+r'.gpickle') for graph in ["dg","ug","eg"]]
    graphs_paths = [os.path.join(root_cs, graph_name) for graph_name in graphs_names]


    dg, ug, eg = get_graphs(graphs_paths)
    dg2 = get_dg2vvday(root_cs)

    # test que dg2 est bien construit
    if len(set(dg2.nodes) - set(dg.nodes)) > 0:
        print(sequence)
    # (en fait, pour trois séquences, ça a merdouillé
    # - sans doute manip de decomp. dans ce cas, on ne ug2 = ug)

    if len(nx.transitive_closure(dg2).edges) < len(nx.transitive_closure(dg).edges):
        print(sequence)
        ug2 = ug
        dg2 = dg
        save_dg2vvday(dg2, root_cs)
        save_ug2vvday(ug2, root_cs)

    else:
        tdg = nx.transitive_closure(dg)
        tdgu = tdg.to_undirected()
        ug2_ = nx.complement(tdgu)

        # vérif que ug2_ contient bien ug (aux auto edges près)
        for e in ug.edges:
            if not ug2_.has_edge(e[0], e[1]) and (e[0] != e[1]):
                print('aië')


        # ajout des edges de ug2_
        new_ug_edges = set([(e[0], e[1], -1) for e in ug2_.edges\
                         if not ug.has_edge(*e)])
        counter += len(new_ug_edges)
        print(len(new_ug_edges))

        ug.add_weighted_edges_from(new_ug_edges)

        save_ug2vvday(ug, root_cs)

print(counter)

#%% Vérification que les arêtes additionnelles
# sont correctes:

liste_verif = [('00010618_6_20161209_150114.jpg', '00010618_6_20161201_103013.jpg', -1),
('00010621_0_20120109_143302.jpg', '00010621_0_20140621_100310.jpg', -1),
('00010628_7_20120107_090956.jpg', '00010628_7_20120107_080955.jpg', -1),
('00010628_7_20120107_090956.jpg', '00010628_7_20120107_073955.jpg', -1),
('00010628_7_20120107_073955.jpg', '00010628_7_20120107_080955.jpg', -1),
('00010657_7002_20130925_103845.jpg', '00010657_7002_20130721_180838.jpg', -1),
('00010660_36_20140417_054134.jpg', '00010660_36_20140524_184135.jpg', -1),
('00010726_5_20150518_091628.jpg', '00010726_5_20150415_174635.jpg', -1),
('00010853_0_20100812_170528.jpg', '00010853_0_20110128_095112.jpg', -1),
('00010853_0_20110508_142110.jpg', '00010853_0_20110710_085110.jpg', -1),
('00010871_0_20110120_123852.jpg', '00010871_0_20110126_113854.jpg', -1),
('00010874_19_20131211_121201.jpg', '00010874_19_20131220_124158.jpg', -1),
('00010926_53_20140521_180312.jpg', '00010926_53_20140609_173316.jpg', -1),
('00011287_0_20100504_022356.jpg', '00011287_0_20100401_201949.jpg', -1),
('00011287_0_20100305_211757.jpg', '00011287_0_20100522_171919.jpg', -1),
('00011351_1_20150121_220130.jpg', '00011351_1_20161005_210131.jpg', -1),
('00011351_1_20150121_220130.jpg', '00011351_1_20170107_170127.jpg', -1),
('00011377_2_20100718_170031.jpg', '00011377_2_20100531_140021.jpg', -1),
('00011385_10001_20150418_120436.jpg', '00011385_10001_20150430_153441.jpg', -1),
('00011385_10001_20150418_120436.jpg', '00011385_10001_20150504_203437.jpg', -1)]
compas = []
for e in liste_verif:
    name0 = e[0]
    name1 = e[1]
    sequence = e[0].split('_')[0] + '_' + e[0].split('_')[1]
    dataset = os.path.join(root_dataset, sequence)
    images_dataset = join(dataset, 'images')
    compas.append(compare(name0, name1, mode, images_dataset, critere))
#%% images fully noisy

root = r"D:\sniff_webcams\AMOS"

dir_noisy_images = os.path.join(root,
                          r"full_noisy_images")



# 1: Ajout de toutes les images annotées 'f' dans bamosvv2 dans le pool
for name_img in corrected_lbls:
    if 'f' in corrected_lbls[name_img]['levelvv']:
        sequence = name_img.split('_')[0] + '_' + name_img.split('_')[1]
        seq = corrected_lbls[name_img]['sequence']
        src = join(dir_splitted, seq, 'images', name_img)
        dst = join(dir_noisy_images, name_img)
        shutil.copy(src,dst)

# à la main: sélection des images annotées 'f' universelles (ie: ne comportant aucune info, et ne correspondant pas à un changement de scène)
# -> nuit noires, et surtout caméras "bouchées".
#%% Construction de BAMOSvv3: images/graphe
dir_bamos = os.path.join(root,
                            'BAMOSvv3')
dir_bamos_images = join(dir_bamos, 'images')

for sequence in os.listdir(dir_splitted):
    dir_sequence = join(dir_splitted, sequence, 'images')


    images = os.listdir(dir_sequence)

    for name_image in images:
        src = join(dir_sequence, name_image)
        dst = join(dir_bamos_images, name_image)
        shutil.copy(src, dst)
# 5374 images
#%% 2) pool_BAMOSvv2 -> BAMOSvv3


dir_pool_BAMOSvv = os.path.join(root,
                            'pool_BAMOSvv2')

for sequence in os.listdir(dir_pool_BAMOSvv):
    dir_pool_images = join(dir_pool_BAMOSvv, sequence, 'images')
    dir_sequence = join(dir_splitted, sequence, 'images')

    try:
        images = os.listdir(dir_sequence)
    except:
        images = []

    for name_image in os.listdir(dir_pool_images):
        if (not 'Copie' in name_image) and (name_image not in images):
            src = join(dir_pool_images, name_image)
            dst = join(dir_bamos_images, name_image)
            shutil.copy(src, dst)

#%% 3) full noisy images -> BAMOSvv3
dir_noisy_images = os.path.join(root,
                          r"full_noisy_images")

for name_image in os.listdir(dir_noisy_images):
    src = join(dir_noisy_images, name_image)
    dst = join(dir_bamos_images, name_image)
    shutil.copy(src, dst)

# 41736 images
#%% Construction des graphs complets:

# 1) Aggrégation des graphes venant de BAMOSvv2:

tdg = nx.DiGraph()
tug = nx.Graph()
teg = nx.Graph()

for sequence in os.listdir(dir_splitted):
    try:
        dir_labels = os.path.join(dir_splitted,sequence,'labels_ord')
        # attention: poset_vvday a perdu les poids. Mais ils sont dans dg_vv
        # qui contient la même information
        dg_path = os.path.join(dir_labels, 'dg_vvday.gpickle')
        ug_path = os.path.join(dir_labels, 'posetbar_vvday.gpickle')
        eg_path = os.path.join(dir_labels, 'eg_vvday.gpickle')

        local_dg = nx.read_gpickle(dg_path)
        local_ug = nx.read_gpickle(ug_path)
        local_eg = nx.read_gpickle(eg_path)
    #            print(len(local_ug.edges))

        tdg = nx.compose(tdg, local_dg)
        tug = nx.compose(tug, local_ug)
        teg = nx.compose(teg, local_eg)


    except:
        pass
        print('nothing in sequence: ' + str(sequence))



#%% Contrôle:

print(len(tdg.edges), len(tug.edges), len(teg.edges))


#washing
two_cycles = set(tdg.edges).intersection(invert_edges(set(tdg.edges)))
two_cycles = edges_and_inverted_edges(two_cycles)
print(str(len(two_cycles)) + ' 2-cycles')


tdg.remove_edges_from(two_cycles)
tdg = nx.transitive_closure(tdg)
two_cycles = set(tdg.edges).intersection(invert_edges(set(tdg.edges)))
two_cycles = edges_and_inverted_edges(two_cycles)
print(str(len(two_cycles)) + ' 2-cycles after washing')


vs03 = edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3')


vs03eg = {edge for edge in vs03 if edge in teg.edges}
vs03 = edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
tdg.remove_edges_from(vs03)
tug.remove_edges_from(vs03)
teg.remove_edges_from(vs03eg)
print(str(len(vs03)) + ' 0vs3 after washing')
print(len(tdg.edges), len(tug.edges), len(teg.edges))


# 26841 12489 3373
#%% Ajout des edges "noisy"
images = sorted(os.listdir(dir_bamos_images))
new_edges = []
for noisy_image in os.listdir(dir_noisy_images):
    sequence = noisy_image.split('_')[0]
    same_seq_images = [img for img in images if img.split('_')[0] == sequence]
    new_edges += [(noisy_image, img, -2) for img in same_seq_images]

print(len(new_edges)) #141138
tug.add_weighted_edges_from(new_edges)
#%% contrôle
vs03 = edges_and_inverted_edges(tug.edges).intersection(set(tdg.edges))
print(str(len(vs03)) + ' 0vs3')
tdg.remove_edges_from(vs03)
#%%
print(len(tdg.edges), len(tug.edges), len(teg.edges))
#26839 150096 3373
#%% Enregistrement des graphes

path_tdg = os.path.join(dir_bamos, 'tdg_bamosvv_041123')
path_tug = os.path.join(dir_bamos, 'tug_bamosvv_041123')
path_teg = os.path.join(dir_bamos, 'teg_bamosvv_041123')

nx.write_gpickle(tdg, path_tdg)
nx.write_gpickle(tug, path_tug)
nx.write_gpickle(teg, path_teg)
